# Event Selection Example

This notebook gives some examples of how to do event selections in the columnar framework with Awkward Array. As in the other notebooks, make sure that you've run the appropriate `voms-proxy-init` command in order to access the file for this notebook.

In [3]:
import awkward as ak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection

redirector = "root://cmsxrootd.fnal.gov//"
filename = redirector+"/store/mc/RunIISummer20UL17NanoAODv9/Z2JetsToNuNu_M-50_LHEFilterPtZ-400ToInf_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v2/2510000/012B57B0-38DB-8246-B875-2E5E4625C134.root"
events = NanoEventsFactory.from_root(
        filename,
        schemaclass=NanoAODSchema.v6,
    ).events()
print("Events loaded")

Events loaded


If we want to just pull out a certain subset of events by number, we can do that by slicing.

In [6]:
print(f"There are {len(events)} total events in the file")
fewer_events = events[10:15]
print(f"In our fewer_events, there are {len(fewer_events)} events")

There are 550083 total events in the file
In our fewer_events, there are 5 events


To filter out events, we can use a 1D array of Booleans of the same length as the number of events.

For example, we can ask for events with at least 3 AK4 jets. To see the number of jets per event, we use `ak.num`.

In [9]:
print(f"The number of jets per event: {ak.num(fewer_events.Jet)}")
jet3_filt = ak.num(fewer_events.Jet) >= 3
print(f"The filter for >= 3 jets: {jet3_filt}")
fewer_events_jet3 = fewer_events[jet3_filt]
print(f"The number of jets in remaining events: {ak.num(fewer_events_jet3.Jet)}")

The number of jets per event: [4, 4, 2, 6, 7]
The filter for >= 3 jets: [True, True, False, True, True]
The number of jets in remaining events: [4, 4, 6, 7]


We could use this method for all of our event selection, but coffea provides a more memory-efficient way to do event selections. This is the coffea.analysis_tools.PackedSelection class.

We'll create some 1D arrays of Booleans like above, and then add them to the PackedSelection object with a name. We can then apply different combinations of selections to our events.

In [10]:
selection = PackedSelection()
selection.add("Nelectrons>=2",ak.num(events.Electron)>=2)
selection.add("Nmuons>=2",ak.num(events.Muon)>=2)

This added two separate selections, one of which asks for at least 2 electrons, and one which asks for at least 2 muons in the event.

We can also use High Level Trigger flags for selections. Here, we use a dielectron and a dimuon trigger.

In [11]:
selection.add("eeTrigger", events.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ)
selection.add("mumuTrigger",events.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8)

We can use the `all()` method of the PackedSelection to require that events pass $\textbf{all}$ of the selections we give to the method. We can use this directly to filter, or we can use it to add new selections that are just combinations of other selections.

In [12]:
selection.add("dielectron",selection.all("eeTrigger","Nelectrons>=2"))
selection.add("dimuon",selection.all("mumuTrigger","Nmuons>=2"))
muEvents = events[selection.all("dimuon")]
eleEvents = events[selection.all("dielectron")]
print(f"We started with {len(events)} events")
print(f"We have {len(muEvents)} that pass the dimuon requirements")
print(f"We have {len(eleEvents)} that pass the dielectron requirements")

We started with 550083 events
We have 186 that pass the dimuon requirements
We have 47 that pass the dielectron requirements
